# E-commerce Sales Analysis


#### Import libraries

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
import plotly
import random
import calendar
from datetime import datetime
# from scipy.interpolate import make_interp_spline, BSpline

import json
from urllib.request import urlopen

pd.set_option('display.max_columns', None) # display full columns


### Dataset Overview

In [ ]:
df = pd.read_csv('olist_dataset(full).csv')
df

In [ ]:
df.describe()

In [ ]:
df.info()


### Cleaning and preparing data

#### Datetime format

In [ ]:
# df['timestamp'] = df["timestamp"].apply(lambda date : str(date).replace('/','-'))
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

#### Create the state map and its convert func

In [ ]:
df_state_interpret = pd.DataFrame({"state":['AC','AL','AM','AP','BA','CE','DF','ES','GO','MA','MG','MS','MT','PA','PB','PE','PI','PR','RJ','RN','RO','RR','RS','SC','SE','SP','TO'],
                                            "full_state": ['Acre','Alagoas','Amazonas','Amapá','Bahia','Ceará','Distrito Federal','Espírito Santo','Goiás','Maranhão','Minas Gerais','Mato Grosso do Sul ','Mato Grosso','Pará','Paraíba','Pernambuco','Piauí','Paraná','Rio de Janeiro','Rio Grande do Norte','Rondônia','Roraima','Rio Grande do Sul  ','Santa Catarina','Sergipe','São Paulo','Tocantins']})
def ConvertState(eri) -> pd.Series:
    return pd.merge(df_state_interpret, eri, on='state', how='inner')['full_state']


#### Normalize and reduce id from 32 -> 11 hashed chars

In [ ]:
df['order_id'] = df['order_id'].str[::3]
df['customer_id'] = df['customer_id'].str[::3]
df['product_id'] = df['product_id'].str[::3]
df['seller_id'] = df['seller_id'].str[::3]
df.head()

#### New dataset

In [ ]:
df.info()

###  Exploratory Data Analysis

#### Distribution of quantitative variables (numerical data)

In [ ]:

quantitative = [c for c in df.columns if df.dtypes[c] != 'object']
quantitative.remove('order_purchase_timestamp')
quantitative.remove('rating')
quantitative.remove('quantity')
quantitative


In [ ]:

rows, cols = 2,3

plt.figure(figsize=(20, 10))
plt.subplots_adjust(hspace=0.5, wspace=0.5)

for i, col in enumerate(quantitative):
    plt.subplot(rows, cols, i + 1)
    sns.histplot(data=df, x=col, kde=True,bins=30,  edgecolor='black')
    plt.xlabel(col)  
    plt.ylabel('Density')  
    plt.title(f'variable: {col}')  

plt.tight_layout()
plt.show()


#### Distribution of qualitative variables (categorical data)

In [ ]:

qualitative = [c for c in df.columns if df.dtypes[c] == 'object']
qualitative = list(filter(lambda  x: not str(x).__contains__('id'), qualitative))
qualitative.append('rating')

qualitative

##### Rating, payment type and order  status

In [ ]:

rows, cols = 2,3

plt.figure(figsize=(20, 10))
plt.subplots_adjust(hspace=0.5, wspace=0.5)

plt.subplot(rows, cols, 4)
# region Plot 1 for rating

labels = ['1 star', '2 star', '3 star', '4 star', '5 star']

title ="Rating"
rating = df['rating'].value_counts().sort_index().to_frame()
data = rating['count'].values

Spring_Pastels = ["#fd7f6f", "#7eb0d5", "#b2e061", "#bd7ebe", "#ffb55a", "#ffee65", "#beb9db", "#fdcce5", "#8bd3c7"]

def func(pct, allvalues):
    absolute = int(pct / 100.*np.sum(allvalues))
    return "{:.1f}%\n({:d} )".format(pct, absolute)

tuple = plt.pie(
    x=data, 
    labels=labels,

    autopct=lambda pct: func(pct, data),
    # autopct='%1.2f%%', # show percentage with two decimal points
    textprops={'fontsize':11}, # increase the size of all text elements 
    colors=Spring_Pastels[:len(data)],

    startangle=90, 
    counterclock=False,
    
    labeldistance=1.2,
    pctdistance=0.7,

    explode = list(map(lambda x : x if (x < 0.15) else 0.01, np.random.random(len(data))))
)

for text in tuple[1]: #texts
    text.set_fontweight('regular')
    text.set_horizontalalignment('center')

for auto_text in tuple[2]: # Autotexts
    auto_text.set_horizontalalignment('center')
    auto_text.set_fontstyle('italic')

plt.legend(tuple[0], labels,
        title="Type",
        loc="center left",
        bbox_to_anchor=(1, 0, 0, 1.5))
plt.title(
    label=title, # Label
    fontdict={"fontsize":17},
    loc='left',
    color='c'
)
# endregion

plt.subplot(rows, cols, 1)

# region Plot 2 for payment type
data = df['payment_type'].value_counts().sort_values()
labels = data.index
title = "Payment types"

x_axis_name = "No. orders"
y_axis_name = "Payment types"

plt.barh(y=labels,width=data.values, color ='slateblue',edgecolor ='grey',height=0.5, label ='Payment type') 
for i in range(len(data)):
    plt.text(data.values[i] + 5000, i,data.values[i], ha='center')
plt.grid(True, linewidth=0.5, linestyle='--') 
plt.xlabel(x_axis_name,fontdict={'color':'deepskyblue','family':'serif','size':15}, labelpad=10)
plt.ylabel(y_axis_name,fontdict={'color':'deepskyblue','family':'serif','size':15},labelpad=10)
plt.title(title,fontdict={'color':'royalblue','family':'serif','size':17},loc='center',pad=20)
plt.xlim(1, 90000)
plt.legend(loc='lower right')
# endregion

plt.subplot(rows, cols, 2)

# region Plot 3 for order status
data = df['order_status'].value_counts().sort_values()
labels = data.index
title = "Order status"
y_axis_name = "All order status"
x_axis_name = "N0. orders"

plt.barh(y=labels,width=data.values, color ='dodgerblue', edgecolor ='grey', label ='Status') 
for i in range(len(data)):
    plt.text(data.values[i]+ 6000 , i,data.values[i], ha='center')

plt.xlabel(x_axis_name,fontdict={'color':'deepskyblue','family':'serif','size':15}, labelpad=10)
plt.ylabel(y_axis_name,fontdict={'color':'deepskyblue','family':'serif','size':15},labelpad=10)
plt.title(title,fontdict={'color':'royalblue','family':'serif','size':17},loc='center',pad=20)
plt.legend(loc='lower right')
plt.xlim(0, 120000)

# endregion

plt.tight_layout()
plt.show()



##### Customer state and Seller state

In [ ]:
df_cus_state = pd.DataFrame()
df_cus_state['no. customers'] = df['customer_id'].copy()
df_cus_state['state'] = df['customer_state'].copy()
df_cus_state = df_cus_state.drop_duplicates(['no. customers', 'state'])
df_cus_state = df_cus_state.groupby(by='state', sort=True).count()

df_seller_state = pd.DataFrame()
df_seller_state['no. sellers'] = df['seller_id'].copy()
df_seller_state['state'] = df['seller_state'].copy()
df_seller_state = df_seller_state.drop_duplicates(['no. sellers', 'state'])
df_seller_state = df_seller_state.groupby(by='state', sort=True).count()

df_seller_cus = pd.merge(df_cus_state,
                  df_seller_state,
                  on='state',how='left').fillna(0)
df_seller_cus = df_seller_cus.reset_index()

#region Plotting
plt.figure(figsize=(15,8))
plt.subplot(2,1,1)
plt.title("No. Customer and seller in each state", pad=20, fontsize=18)

sns.barplot(data=df_seller_cus, x='state', y='no. customers',palette='Set2', hue='state', errorbar=None)
for i in range(len(df_seller_cus['no. customers'])):
    plt.text(i, df_seller_cus['no. customers'][i] + 600, df_seller_cus['no. customers'][i], ha='center')
plt.ylim(0,45000)
plt.grid(axis='both',linestyle='--',linewidth=0.5,color='silver')
plt.xlabel(" ")
plt.subplot(2,1,2)
sns.barplot(data=df_seller_cus, x='state', y='no. sellers',palette='Set2', hue='state', errorbar=None)
for i in range(len(df_seller_cus['no. sellers'])):
    plt.text(i, df_seller_cus['no. sellers'][i] + 50, df_seller_cus['no. sellers'][i], ha='center')
plt.ylim(0,2000)

plt.grid(axis='both',linestyle='--',linewidth=0.5,color='silver')

plt.show()

#endregion


## Segmentation

#### Customer and seller segmentation

In [ ]:
df_cus_payment = df[['customer_id', 'quantity', 'price', 'freight_value']].copy()
df_cus_payment['payment'] = ( df_cus_payment['price'] + df_cus_payment['freight_value'] ) * df_cus_payment['quantity']
df_cus_payment = df_cus_payment[['customer_id', 'payment']].groupby('customer_id')['payment'].sum().reset_index()
df_cus_payment

df_sel_revenue = df[['seller_id', 'quantity', 'price']].copy()
df_sel_revenue['revenue'] = ( df_sel_revenue['price'] ) * df_sel_revenue['quantity']
df_sel_revenue = df_sel_revenue[['seller_id', 'revenue']].groupby('seller_id')['revenue'].sum().reset_index()
df_sel_revenue

labels = ["Vip \n(>= 1000 BRL)","Diamond \n(500 < 1000) BRL)", "Silver \n(200 < 500) BRL)","Bronze \n(50 < 200) BRL)", "Lowtime \n(<=50 BRL)"]
data = pd.DataFrame(data={"No. customers": [df_cus_payment[(df_cus_payment["payment"] >= 1000)].shape[0], 
                            df_cus_payment[(df_cus_payment["payment"] >= 500) & (df_cus_payment["payment"] < 1000)].shape[0],
                            df_cus_payment[(df_cus_payment["payment"] >= 200) & (df_cus_payment["payment"] < 500)].shape[0],
                            df_cus_payment[(df_cus_payment["payment"] >= 50) & (df_cus_payment["payment"] < 200)].shape[0],
                            df_cus_payment[df_cus_payment["payment"].between(0, 50)].shape[0]]}, index=[1,2,3,4,5])

title = "Customer segmentation based on total amount of spending"
x_axis_name = "Customer segments"
y_axis_name = "Total spending"

plt.figure(figsize=(13,7), dpi=100)
plt.subplots_adjust(hspace=0.7, wspace=0.7)
plt.subplot(1,2,1)

# region Plotting Bar Char
plt.bar(labels, data['No. customers'], width=0.3,label="Customer",color=sns.color_palette('plasma', len(data))) 

for i in range(1,len(data['No. customers']) + 1):
    plt.text(i - 1, data['No. customers'][i] + 500,data['No. customers'][i], ha='center')

plt.grid(True, linewidth=0.2, linestyle='--',color='c') #only for single column
plt.xlabel(x_axis_name,fontdict={'color':'deepskyblue','family':'serif','size':15}, labelpad=10)
plt.ylabel(y_axis_name,fontdict={'color':'deepskyblue','family':'serif','size':15},labelpad=10)
plt.title(title,pad=10)
plt.legend(loc='upper left')
plt.xticks(rotation=60)


# endregion

plt.subplot(1,2,2)

df_sel_revenue = pd.DataFrame({'Rank': ['A (>50000)','B (20000, 50000)','C (50000, 20000)','D (1000, 5000)','E (<1000)'],
              'No. seller': [
                  df_sel_revenue[df_sel_revenue['revenue'] > 50000].shape[0],
                  df_sel_revenue[df_sel_revenue['revenue'].between(20000, 50000)].shape[0],
                  df_sel_revenue[df_sel_revenue['revenue'].between(5000, 20000)].shape[0],
                  df_sel_revenue[df_sel_revenue['revenue'].between(1000, 5000)].shape[0],
                  df_sel_revenue[df_sel_revenue['revenue'].between(0, 1000)].shape[0],
              ]})

labels = df_sel_revenue['Rank']
data = df_sel_revenue
title = "Seller Raking based on total revenue"
x_axis_name = "Seller Rank"
y_axis_name = "Total revenue"

# region Plotting seller ranking
plt.bar(labels, data['No. seller'], width=0.3,label="Sellers",color=sns.color_palette('plasma', len(data))) 

for i in range(len(data['No. seller'])):
    plt.text(i, data['No. seller'][i] + 20,data['No. seller'][i], ha='center')

plt.grid(True, linewidth=0.2, linestyle='--',color='c') #only for single column
plt.xlabel(x_axis_name,fontdict={'color':'deepskyblue','family':'serif','size':15}, labelpad=10)
plt.ylabel(y_axis_name,fontdict={'color':'deepskyblue','family':'serif','size':15},labelpad=10)
plt.title(title,loc='center',pad=10)
plt.legend()
plt.xticks(rotation=60)

plt.tight_layout()
plt.show() 
# endregion



#### Top 5 categories has the highest no. product versus the others one?

In [ ]:
df_pro_cate = df[['category', 'product_id']].groupby('category').nunique()
df_pro_cate = df_pro_cate.reset_index()
df_pro_cate.loc[len(df_pro_cate.index)] = ['Others cate', (df_pro_cate['product_id'].sum() - df_pro_cate['product_id'].head().sum())] 
df_pro_cate = df_pro_cate.sort_values('product_id', ascending=False).head(6)
df_pro_cate = df_pro_cate.reset_index(drop=True)
df_pro_cate

labels = df_pro_cate['category']
data = df_pro_cate['product_id']
x_axis_name = "No. products"
y_axis_name = "Categories"

Spring_Pastels = ["#fd7f6f", "#7eb0d5", "#b2e061", "#bd7ebe", "#ffb55a", "#ffee65", "#beb9db", "#fdcce5", "#8bd3c7"]

# region Plot 1
def func(pct, allvalues):
    absolute = int(pct / 100.*np.sum(allvalues))
    return "{:.1f}%\n({:d})".format(pct, absolute)
 

fig, ax = plt.subplots(1,2,figsize=(13, 7))
wedges, texts, autotexts = ax[0].pie(x=data,
                                  autopct=lambda pct: func(pct, data),
                                  explode=list(map(lambda x : x if (x < 0.24) else 0.08, np.random.random(len(data))))   ,
                                  labels=labels,
                                  shadow=True,
                                  colors=(Spring_Pastels)[:len(data)],
                                  startangle=180,
                                  wedgeprops= {'linewidth': 1, 'edgecolor': "slategray"}, # border
                                  textprops=dict(color="black"))
ax[0].legend(wedges, labels,
          title="Cars",
          loc="center left",
          bbox_to_anchor=(0, 0, 0, 0))

plt.setp(autotexts, size=10, )
ax[0].set_title("Top  5 categories with the highest no. products", pad=20)
# endregion 


df_seller_cate = df[['category', 'seller_id']].groupby('category').nunique()
df_seller_cate = df_seller_cate.reset_index()
df_seller_cate.loc[len(df_seller_cate.index)] = ['Others cate', 
                                                 (df_seller_cate['seller_id'].sum() - df_seller_cate['seller_id'].head().sum())] 
df_seller_cate = df_seller_cate.sort_values('seller_id', ascending=False).head(6)
df_seller_cate = df_seller_cate.reset_index(drop=True)
df_seller_cate

labels = df_seller_cate['category']
data = df_seller_cate['seller_id']
x_axis_name = "No. sellers"
y_axis_name = "Categories"

# region Plot 2
wedges, texts, autotexts = ax[1].pie(x=data,
                                  autopct=lambda pct: func(pct, data),
                                  explode=list(map(lambda x : x if (x < 0.24) else 0.08, np.random.random(len(data))))   ,
                                  labels=labels,
                                  shadow=True,
                                  colors=sns.color_palette('Set2', len(data)),
                                  startangle=180,
                                  wedgeprops= {'linewidth': 1, 'edgecolor': "slategray"}, # border
                                  textprops=dict(color="black"))
ax[1].legend(wedges, labels,
          title="Category",
          loc="center left",
          bbox_to_anchor=(0, 0, 0, 0))

plt.setp(autotexts, size=10, )
ax[1].set_title("Top  5 categories with the highest no. seller", pad=20)
# endregion
plt.axis('off')
plt.tight_layout()
plt.show()



#### Average original price of products in each state

In [ ]:
df_ave_price_state = df.groupby(['seller_state']).mean('price')[[ 'price']]
df_ave_price_state['price'] = round(df_ave_price_state['price'], 2)
df_ave_price_state = df_ave_price_state.reset_index()
df_ave_price_state = df_ave_price_state.rename(columns={'seller_state': 'state'})

plt.figure(figsize=(16,7))
plt.title("Average max retail price in each state", pad=20, fontsize=17)

sns.barplot(data=df_ave_price_state, x='state', y='price',palette='viridis', hue='state', errorbar=None)
for i in range(len(df_ave_price_state)):
    plt.text(i, df_ave_price_state['price'][i] + 10, df_ave_price_state['price'][i], ha='center')
# plt.xticks(rotation=60)/
# plt.axis('off')
# plt.xticks([])
# plt.yticks([])
plt.show()

### The no. product was bought in each state and the no. product in it

In [ ]:
df_pro_sell_state =  pd.merge(
    df[['customer_state', 'product_id']].groupby('customer_state').
    nunique().reset_index().rename(columns={'customer_state':'state', 'product_id':'no. pro purchased'}),
    df[['seller_state', 'product_id']].groupby('seller_state').
    nunique().reset_index().rename(columns={'seller_state':'state', 'product_id':'no. products'}),
    on='state',how='left').fillna(0)
df_pro_sell_state['no. products'] = df_pro_sell_state['no. products'].astype('int32')

# region Plotting
plt.figure(figsize=(15,8))
plt.subplot(2,1,1)
plt.title("No. product was bought and no. product sold in each state", pad=20, fontsize=18)

sns.barplot(data=df_pro_sell_state, x='state', y='no. products',palette='viridis_r', hue='state', errorbar=None)
for i in range(len(df_pro_sell_state['no. products'])):
    plt.text(i, df_pro_sell_state['no. products'][i] + 1000 , df_pro_sell_state['no. products'][i], ha='center')
plt.ylim(0,26000)
plt.grid(axis='both',linestyle='-',linewidth=0.3,color='silver')
plt.xlabel(" ")

plt.subplot(2,1,2)
sns.barplot(data=df_pro_sell_state, x='state', y='no. pro purchased',palette='viridis_r', hue='state', errorbar=None)
for i in range(len(df_pro_sell_state['no. pro purchased'])):
    plt.text(i, df_pro_sell_state['no. pro purchased'][i] + 1000 , df_pro_sell_state['no. pro purchased'][i], ha='center')
plt.ylim(0,22000)
plt.grid(axis='both',linestyle='-',linewidth=0.3,color='silver')
plt.show()
# endregion Plotting



## Time series

### No. domestic orders growth

In [ ]:
df_gross_domestic = df[['order_purchase_timestamp', 'order_id' ]]
df_gross_domestic.loc[:, 'order_purchase_timestamp'] = df_gross_domestic['order_purchase_timestamp'].dt.date

df_gross_domestic = df_gross_domestic.groupby('order_purchase_timestamp').count().reset_index()

df_gross_domestic.sort_values('order_id').tail(10)

plt.figure(figsize=(13,7))

plt.plot(
    df_gross_domestic['order_purchase_timestamp'],
    df_gross_domestic['order_id'],
    label='no. orders')

plt.annotate('Day : 2017-11-24\nNo. orders : 1158', 
             xy=(pd.to_datetime('2017-11-27'), 1150 ), 
             xytext=(pd.to_datetime('2017-06-01'), 1000 ),
             fontsize=10, fontweight="regular", ## Font settings
             arrowprops=dict(arrowstyle="<-", facecolor="black", connectionstyle="arc3, rad=-0.5",
                             shrinkA=10, shrinkB=10),
                             bbox=dict(boxstyle="round",   ## Bounding box settings
                                        facecolor="tomato", alpha=0.6, linewidth=0.5))
plt.annotate('Day : 2017-11-25\nNo. orders : 491', 
             xy=(pd.to_datetime('2017-11-30	'), 430 ), 
             xytext=(pd.to_datetime('2018-02-01'), 800 ),
             fontsize=10, fontweight="regular", ## Font settings
             arrowprops=dict(arrowstyle="<-", facecolor="black", connectionstyle="arc3, rad=0.4",
                             shrinkA=10, shrinkB=10),
                             bbox=dict(boxstyle="round",   ## Bounding box settings
                                        facecolor="tomato", alpha=0.5, linewidth=0))
plt.annotate('Day : 2018-08-07\nNo. orders : 364', 
             xy=(pd.to_datetime('2018-08-07'), 350 ),
             xytext=(pd.to_datetime('2018-03-25'), 600 ),
             fontsize=10, fontweight="regular", ## Font settings
             arrowprops=dict(arrowstyle="<-", facecolor="black", connectionstyle="arc3, rad=-0.6",
                             shrinkA=10, shrinkB=10),
                             bbox=dict(boxstyle="round",   ## Bounding box settings
                                        facecolor="tomato", alpha=0.5, linewidth=0.1))
plt.xlabel('Date')
plt.grid(axis='both',linestyle='--',linewidth=0.5,color='silver') # đường Lưới
plt.ylabel('Value')
plt.ylim(-100, 1300)
plt.title("No. domestic order growth 2016-2018 ",pad=20,fontsize=18)
plt.legend()
plt.show()


### Monthly Revenue

In [ ]:
df_monthly_revenue = df[['order_purchase_timestamp','quantity', 'price', 'freight_value']].copy()
df_monthly_revenue['payment'] = (df_monthly_revenue['price'] + df_monthly_revenue['freight_value']) * df_monthly_revenue['quantity']

df_monthly_revenue_2018 = df_monthly_revenue[df_monthly_revenue['order_purchase_timestamp'].dt.year == 2018]
df_monthly_revenue_2016 = df_monthly_revenue[df_monthly_revenue['order_purchase_timestamp'].dt.year == 2016]
df_monthly_revenue = df_monthly_revenue[df_monthly_revenue['order_purchase_timestamp'].dt.year == 2017]

df_monthly_revenue = df_monthly_revenue.groupby(df_monthly_revenue['order_purchase_timestamp'].dt.date)[['payment']].mean().reset_index()
df_monthly_revenue_2016 = df_monthly_revenue_2016.groupby(df_monthly_revenue_2016['order_purchase_timestamp'].dt.date)[['payment']].mean().reset_index()
df_monthly_revenue_2018 = df_monthly_revenue_2018.groupby(df_monthly_revenue_2018['order_purchase_timestamp'].dt.date)[['payment']].mean().reset_index()

df_monthly_revenue_2018['order_purchase_timestamp'] = df_monthly_revenue_2018['order_purchase_timestamp'].astype('str')
df_monthly_revenue_2016['order_purchase_timestamp'] = df_monthly_revenue_2016['order_purchase_timestamp'].astype('str')
df_monthly_revenue['order_purchase_timestamp'] = df_monthly_revenue['order_purchase_timestamp'].astype('str')

df_monthly_revenue_2018['order_purchase_timestamp'] = df_monthly_revenue_2018['order_purchase_timestamp'].str.replace('2018', '2017')
df_monthly_revenue_2016['order_purchase_timestamp'] = df_monthly_revenue_2016['order_purchase_timestamp'].str.replace('2016', '2017')

df_monthly_revenue

plt.figure(figsize=(15,8))


sns.lineplot(data=df_monthly_revenue, x='order_purchase_timestamp', y='payment', label='2017')
sns.lineplot(data=df_monthly_revenue_2016, x='order_purchase_timestamp', y='payment', label='2016')
sns.lineplot(data=df_monthly_revenue_2018, x='order_purchase_timestamp', y='payment', label='2018')
  
pos = [ '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01',  
       '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01', 
       '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01'] 
lab = [ 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'June',  
       'July', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec'] 
plt.xticks( pos, lab) 
  
plt.title('Monthly revenue in three years separately', pad=20)
plt.xlabel( 'Months of the year') 
plt.ylabel('Revenue') 

plt.show()


## Geographic maps

#### Load brazil geography map coordinates

In [ ]:
with urlopen('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson') as response:
    Brazil = json.load(response)
Brazil

### Preparing data for each state in brazil

In [ ]:

df_all_state = pd.merge(df_pro_sell_state,
                        df_ave_price_state,
                        on='state', how='left')
df_cus_state.reset_index(inplace=True)
df_seller_state.reset_index(inplace=True)

df_all_state = pd.merge(df_all_state, df_cus_state, on='state', how='left')
df_all_state = pd.merge(df_all_state, df_seller_state, on='state', how='left').fillna(0)

df_all_state['state'] = ConvertState(df_all_state['state'])
df_all_state['price'] = df_all_state['price'].astype('str')
df_all_state['price'] = df_all_state['price'] + ' BRL'

df_all_state


In [ ]:

# fig = px.choropleth(df_all_state, #soybean database
# locations = 'state', #define the limits on the map/geography
# hover_name = 'state', #the information in the box
# geojson = Brazil, #shape information
# color = "no. customers", #defining the color of the scale through the database
# hover_data =["no. customers","no. sellers","no. products", "no. pro purchased", "price"],
# title = "All state in brazil",
# )
# fig.update_geos(fitbounds = "locations", visible = False)
# fig.update_layout(coloraxis_colorbar=dict(
#     thicknessmode="pixels", thickness=10,
#     lenmode="pixels", len=150,
#     yanchor="top", y=0.8,
#     ticks="outside", ticksuffix=" %",
#     dtick=5
# ))
# fig.update_layout()
# # fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
# fig.show()

### The whole world map

In [ ]:

# fig = px.choropleth_mapbox(
# soybean, #soybean database
# locations = 'Estado', #define the limits on the map/geography
# geojson = Brazil, #shape information
# color = "Produção", #defining the color of the scale through the database
# hover_name = 'Estado', #the information in the box
# hover_data =["Produção","Longitude","Latitude"],
# title = "Produtivida da soja (Toneladas)", #title of the map
# mapbox_style = "carto-positron", #defining a new map style 
# center={"lat":-14, "lon": -55},#define the limits that will be plotted
# zoom = 3, #map view size
# opacity = 0.5, #opacity of the map color, to appear the background
# animation_frame = 'ano' #creating the application based on the yearfig.show()
# )


# plotly.offline.plot(fig, filename = 'Brazil_map.html')

### Correlation